In [62]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
nltk.download('stopwords') 
nltk.download('wordnet')
from nltk.corpus import stopwords

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krist\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\krist\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\krist\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [55]:
# For one text

def keywords(text):
    string = text
    '--------------------------'
    
    def pre_process(string):
        # lowercase
        string =string.lower()
    
        #remove tags
        string =re.sub("</?.*?>"," <> ",text)
    
        # remove special characters and digits
        string =re.sub("(\\d|\\W)+"," ",text)
    
        return string
    
    article = pre_process(string)
    
    '--------------------------'
    #load a set of stop words
    stop_words= stopwords.words("dutch")
    stop_words.extend(['bis', 'NL', 'FR','artikel', "januari", "februari", "maart", "april", "mei", "juni", "juli", "augustus", "september", "oktober", "november", "december"])
    
    #get the text 
    
    docs=[article]  
    
    #create a vocabulary of words, 
    #ignore words that appear in 85% of documents, 
    #eliminate stop words
    cv=CountVectorizer(stop_words=stop_words)
    word_count_vector=cv.fit_transform(docs)
    
    # transform it to tf-idf vector
    
    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
    tfidf_transformer.fit(word_count_vector)
    
    '--------------------------'
    
    def sort_coo(coo_matrix):
        tuples = zip(coo_matrix.col, coo_matrix.data)
        return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

    '--------------------------'
    
    def extract_topn_from_vector(feature_names, sorted_items, topn=10):
        """get the feature names and tf-idf score of top n items"""
    
        #use only topn items from vector
        sorted_items = sorted_items[:topn]

        score_vals = []
        feature_vals = []

        for idx, score in sorted_items:
            fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
            score_vals.append(round(score, 3))
            feature_vals.append(feature_names[idx])

        #create a tuples of feature,score
        #results = zip(feature_vals,score_vals)
        results= {}
        for idx in range(len(feature_vals)):
            results[feature_vals[idx]]=score_vals[idx]
        
        return results
    
    '--------------------------'
    
    # you only needs to do this once
    feature_names=cv.get_feature_names()
    
    result_list=[]
    def get_keywords(x):

        #generate tf-idf for the given document
        tf_idf_vector=tfidf_transformer.transform(cv.transform(x))

        #sort the tf-idf vectors by descending order of scores
        sorted_items=sort_coo(tf_idf_vector.tocoo())

        #extract only the top n; n here is 10
        keys=extract_topn_from_vector(feature_names,sorted_items,10)
        
        result_list.append(list(keys.keys()))
        return result_list
    
    end = get_keywords(docs)
    return end

In [56]:
text = df['Text'][0]

In [59]:
maybe = keywords(text)

In [63]:
def cosine_score(s1):
# Two lists of sentences

    #s1 = ['brusselse overlijden driejarige fiscale behoud begroting belast familiale gunstregime decujus kader bedoeld gaan wetboek regering vervuld bekomen gebleven periode voorwaarden financien overdracht zoals hoofdstedelijke aanving']
    s2 = ['belasting fiscaliteit aanslagjaar inkomsten']

    #Compute embedding for both lists
    embeddings1 = model.encode(s1, convert_to_tensor=True)
    embeddings2 = model.encode(s2, convert_to_tensor=True)

    #Compute cosine-similarits
    cosine_scores = float(util.cos_sim(embeddings1, embeddings2))

    #Output the pairs with their score
    cosine_scores = ("{:.3f}".format(cosine_scores))
    return cosine_scores

In [64]:
cosine_score(maybe)

'0.166'

In [18]:
# Going through a csv file

def keywords_csv(filepath):
    df_idf= pd.read_csv(filepath)
    
    # Removing German translations.
    
    df_idf= df_idf[df_idf["Text"].str.contains("Duitse vertaling")==False]
    
    def pre_process(text):
    
        # lowercase
        text=text.lower()
    
        #remove tags
        text=re.sub("</?.*?>"," <> ",text)
    
        # remove special characters and digits
        text=re.sub("(\\d|\\W)+"," ",text)
    
        return text

    df_idf['Text'] = df_idf['Text'].apply(lambda x:pre_process(x))

    #load a set of stop words
    
    stop_words= stopwords.words("dutch")
    stop_words.extend(['bis', 'NL', 'FR','artikel', "januari", "februari", "maart", "april", "mei", "juni", "juli", "augustus", "september", "oktober", "november", "december"])

    #get the text column 
    
    docs=df_idf['Text'].tolist()
    docs_title=df_idf['Title'].tolist()
    
    #create a vocabulary of words, 
    #ignore words that appear in 85% of documents, 
    #eliminate stop words
    cv=CountVectorizer(max_df=0.85,stop_words=stop_words)
    word_count_vector=cv.fit_transform(docs)
    
    # Call TfidTransformer
    
    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
    tfidf_transformer.fit(word_count_vector)
    
    def sort_coo(coo_matrix):
        tuples = zip(coo_matrix.col, coo_matrix.data)
        return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

    def extract_topn_from_vector(feature_names, sorted_items, topn=10):
        """get the feature names and tf-idf score of top n items"""
    
        #use only topn items from vector
        sorted_items = sorted_items[:topn]

        score_vals = []
        feature_vals = []

        for idx, score in sorted_items:
            fname = feature_names[idx]
        
            #keep track of feature name and its corresponding score
            score_vals.append(round(score, 3))
            feature_vals.append(feature_names[idx])

        #create a tuples of feature,score
        #results = zip(feature_vals,score_vals)
        results= {}
        for idx in range(len(feature_vals)):
            results[feature_vals[idx]]=score_vals[idx]
    
        return results
    
    # you only needs to do this once
    feature_names=cv.get_feature_names()

    # put the common code into several methods
    def get_keywords(idx):

        #generate tf-idf for the given document
        tf_idf_vector=tfidf_transformer.transform(cv.transform([docs[idx]]))

        #sort the tf-idf vectors by descending order of scores
        sorted_items=sort_coo(tf_idf_vector.tocoo())

        #extract only the top n; n here is 10
        keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
        return keywords

       #generate tf-idf for all documents in your list.
    tf_idf_vector=tfidf_transformer.transform(cv.transform(docs))

    results=[]
    for i in range(tf_idf_vector.shape[0]):
    
        # get vector for a single document
        curr_vector=tf_idf_vector[i]
    
        #sort the tf-idf vector by descending order of scores
        sorted_items=sort_coo(curr_vector.tocoo())

        #extract only the top n; n here is 10
        keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
        results.append(list(keywords.keys()))

    df=pd.DataFrame(zip(docs,results),columns=['doc','keywords'])
    
    return df.head()
    
    

In [19]:
keywords_csv('KPMG Tax Case - Scraped Data.csv')

C:\Users\krist\anaconda3\envs\tf\lib\site-packages\sklearn\feature_extraction\text.py:394: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['fr', 'nl'] not in stop_words.
  warnings.warn(
C:\Users\krist\anaconda3\envs\tf\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,doc,keywords
0,nl fr belgiëlex be kruispuntbank wetgeving raa...,"[familiale, successierechten, gunstregime, dri..."
1,nl fr einde eerste woord laatste woord publica...,"[onderwijs, wetenschappen, promotie, specialis..."
2,nl fr belgiëlex be kruispuntbank wetgeving raa...,"[wib, kb, groepsbijdrage, quater, aftrek, xxvi..."
3,nl fr belgiëlex be kruispuntbank wetgeving raa...,"[quater, aanduiding, gedelegeerde, besluit, in..."
4,nl fr einde eerste woord laatste woord publica...,"[vennootschap, startende, aandelen, startersfo..."
